In [1]:
import requests
import requests_cache
requests_cache.install_cache('cache')
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
import json
from pprint import pprint
API_KEY = 'pk.eyJ1IjoibnlvdTA0NSIsImEiOiJjaXJ6bDYxeWcwMXRzMnlwa2tmbzduNmd0In0.gLrTUpqhS3PYYy_AtCp4Gw'
BASE_URL = f"https://api.mapbox.com/search/searchbox/v1/category/services?access_token={API_KEY}&language=en&limit=25"

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
mb = gpd.read_file("statsnz-meshblock-2023-clipped-generalised-GPKG.zip!meshblock-2023-clipped-generalised.gpkg")
mb

,MB2023_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,geometry
0,0000100,12,Mainland,157.497825,157.497825,84526.158926,"MULTIPOLYGON (((172.67990 -34.42180, 172.67991..."
1,0000200,12,Mainland,120.503770,120.503770,122885.186789,"MULTIPOLYGON (((173.01433 -34.39285, 173.01461..."
2,0000300,12,Mainland,7.481859,7.481859,28321.227739,"MULTIPOLYGON (((172.87839 -34.49475, 172.88016..."
3,0000400,12,Mainland,83.342952,83.342952,41499.302475,"MULTIPOLYGON (((172.80881 -34.50158, 172.81036..."
4,0001201,12,Mainland,51.238250,51.238250,62944.884996,"MULTIPOLYGON (((173.06066 -34.70403, 173.06077..."
...,...,...,...,...,...,...,...
56783,4019154,12,Mainland,0.036001,0.036001,1105.893029,"MULTIPOLYGON (((174.80358 -41.25055, 174.80370..."
56784,4019155,12,Mainland,0.019734,0.019734,661.167326,"MULTIPOLYGON (((174.80411 -41.25295, 174.80383..."
56785,4019156,12,Mainland,0.097215,0.097215,1787.390321,"MULTIPOLYGON (((174.79657 -41.25581, 174.79724..."
56786,4019157,12,Mainland,0.066195,0.066195,1083.336705,"MULTIPOLYGON (((174.78952 -41.26050, 174.79035..."


In [3]:
mb["retrieved_at"] = pd.NA
mb["n_POIs"] = pd.NA
mb["bounds"] = mb.bounds.apply(lambda b: f"{b.minx},{b.miny},{b.maxx},{b.maxy}", axis=1)
mb.drop(columns="geometry").to_csv("locations.csv", index=False)

In [4]:
results = {}
for i, bounds in tqdm(mb["bounds"].items(), total=len(mb)):
    try:
        geojson = requests.get(BASE_URL, params={"bbox": bounds}).json()
    except Exception as e:
        print(e)
        break
    for feature in geojson["features"]:
        results[feature["properties"]["mapbox_id"]] = feature
    mb.retrieved_at[i] = pd.Timestamp.now()
    mb.n_POIs[i] = len(geojson["features"])
    if i % 1000 == 0 or i == mb.index.max():
        mb.to_csv("locations.csv", index=False)
        geojson = {"type": "FeatureCollection", "features": list(results.values())}
        with open("data.geojson", "w") as f:
            json.dump(geojson, f)
mb

  0%|          | 0/56788 [00:00<?, ?it/s]

/tmp/ipykernel_2599100/1955391933.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb.retrieved_at[i] = pd.Timestamp.now()
/tmp/ipykernel_2599100/1955391933.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mb.n_POIs[i] = len(geojson["features"])


In [ ]:
pd.Series([r["properties"]["external_ids"].keys() for r in results.values()]).value_counts()

(foursquare, safegraph)    3
(foursquare)               2
dtype: int64